# Steam Games Analysis

In [11]:
from dotenv import load_dotenv
import os
import pymysql
import pandas as pd
from sklearn.neighbors import NearestNeighbors

load_dotenv()

conn = pymysql.connect(host='localhost',
                        port=int(3306),
                        user='root',
                        passwd=os.getenv('MYSQL_PASSWORD'),
                        db='steam')

game_ratings = pd.read_sql('SELECT * FROM vw_games_rating', conn)
user_ratings = pd.read_sql('SELECT * FROM vw_usergamesrating', conn)

In [9]:
game_ratings.sample(5)


,appid,title,genre,price,avg_rating
2128,109700,Achron,Indie,19.99,0.001180
154,253940,Septerra Core,RPG,4.99,0.000587
1359,247660,Deadly Premonition: The Director's Cut,Action,24.99,0.001300
674,294280,Scooby Doo! & Looney Tunes Cartoon Universe: A...,Casual,19.99,0.002200
1430,16120,Plant Tycoon,Casual,9.99,0.138200


In [3]:
user_ratings.sample(5)

,steamid,appid,playtime_forever,tot_playtime_forever,rating,steamid_appid_rnk
172603,76561197971536295,440,267,7625.0,0.0350,5
260042,76561197977761535,16810,106,108534.0,0.0010,91
373035,76561197986365219,237110,14,50446.0,0.0003,130
181074,76561197971984297,49520,4469,225984.0,0.0198,15
293005,76561197980349061,49600,310,214253.0,0.0014,60


In [12]:
#Read The Data
game_ratings['genre'] = game_ratings.genre.astype('category')

#Get User IDs
user_data = user_ratings['steamid'].unique().tolist()

#Get Dummy Variables for Genre
game_genres = game_ratings['genre'].unique().tolist()
del game_genres[12]
genre_dummies = pd.get_dummies(game_ratings['genre'])
game_ratings1 = pd.concat([game_ratings, genre_dummies], axis=1) 
game_ratings1 = game_ratings1.drop(columns=['genre'])

In [13]:
#Recommendation Function by Steam ID (user ID)
def game_recommendation(user_id):
    user_games = user_ratings.loc[user_ratings['steamid'] == user_id]
    user_games = user_games.sort_values(by=['rating'], ascending=False)
    user_appid = user_games['appid'].iat[0]
    your_game = game_ratings1[game_ratings1['appid'] == user_appid]
    your_game = your_game.iat[0,1]
    
    user_top_game = game_ratings1.loc[game_ratings1['appid'] == user_appid]
    user_top_game = user_top_game.drop(columns=['appid'])
    game_ratings2 = game_ratings1.drop(columns=['appid'])
    
    data_train = game_ratings2.drop(columns=['title'])
    data_test = user_top_game.drop(columns=['title'])
    
    model = NearestNeighbors(n_neighbors=10, metric='manhattan', p=1).fit(data_train)
    recs = model.kneighbors(data_test, 10, return_distance=True)  
    rec_ind = recs[1][0]
    rec_ind = rec_ind.tolist()
    
    rec_list = []
    for ind in rec_ind:
        game_title = game_ratings2.loc[[ind],['title']]
        rec_list.append(game_title.iat[0,0])
        
    #Filter Out Games
    user_id = user_data[0]
    played_games = user_ratings[user_ratings['steamid']== user_id]
    played_games = played_games['appid']

    appid_list = []
    num_games = len(played_games)
    for i in range (num_games):
        appid_list.append(played_games[i])

    title_list = []
    for ID in appid_list:
        if ID in game_ratings1['appid']:
            game = game_ratings1[game_ratings1['appid'] == ID]
            game_title = game.iloc[0]['title']
            title_list.append(game_title)
    count = 0      
    for rec in rec_list:
        if rec in title_list:
            rec_list.remove(rec)
            count += 1
        elif rec == your_game:
            rec_list.remove(rec)
            count +=1
    rec_list.sort()

    return your_game, rec_list, count

In [16]:
#Choose to run on a specific user
your_game, rec_list, count = game_recommendation(user_data[500])
print("Your Top Game:", your_game, "\nRecommendations based on Your Top Game:\n", rec_list)
print("Games Omiited: ", count)

Your Top Game: Counter-Strike: Source 
Recommendations based on Your Top Game:
 ['Battlefield: Bad Company™ 2', 'Call of Duty: World at War', 'Call of Duty® 4: Modern Warfare®', 'Call of Duty®: Modern Warfare® 2', 'Call of Duty®: Modern Warfare® 2', 'Far Cry 3', 'Left 4 Dead 2', 'Street Fighter® IV']
Games Omiited:  2


In [ ]:
#Run for all users
for user in user_data:
    your_game, rec_list, count = game_recommendation(user)
    print("Your Top Game:", your_game, "\nRecommendations based on Your Top Game:\n", rec_list)
    print("Games Omiited: ", count)